In [1]:
# Import libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd

# Create a list of page numbers for pagination. The rule is: start at 1 and increment by 50 for each page
page = list(range(1, 600, 50))

# Initialize empty lists to store data
urls = []
names = []
reais = []
cents = []

# Iterate over each page
for number in page:
    url = f"https://lista.mercadolivre.com.br/spray-pimenta_Desde_{number}_NoIndex_True"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

# Find all the products on the page
    sprays = soup.find_all(class_="ui-search-result__wrapper shops__result-wrapper")

    for spray in sprays:

# Find the URL
        url_product = spray.find("a", class_="ui-search-item__group__element shops__items-group-details ui-search-link")
        href = url_product.get("href")
        if href:
            urls.append(href)
            
# Extract the product name
        name_product = spray.find(class_="ui-search-item__title shops__item-title")
        if name_product:
            names.append(name_product.text)

# Find the regular price element, not the price in credit card
        regular_price = spray.find(class_="andes-money-amount ui-search-price__part shops__price-part andes-money-amount--cents-superscript")
        if regular_price:
# Find the price text and check if it contains cents
            reales_price = regular_price.find(class_="andes-money-amount__fraction")
            if reales_price:
                reais.append(reales_price.text)            
                cents_price = regular_price.find(class_="andes-money-amount__cents andes-money-amount__cents--superscript-24")
                if cents_price:
                    cents.append(cents_price.text)
                else:
                    cents.append("00")

# Create a dictionary with the extracted data
items = {"url": urls, "name": names, "reais": reais, "cents": cents}

# Create a DataFrame
df = pd.DataFrame(items)

# Save the DataFrame
df.to_csv("data_spray.csv", index=False)